In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import medleydb
from pitch_tracker.utils.medleydb_melody import compute_end_time_stamps, add_sequence_to_melody, make_blank_melody_sequence
from pitch_tracker.utils.files import list_all_file_paths_in_dir
from pitch_tracker.utils import dataset

from tqdm import tqdm


In [21]:

NOTEBOOK_DIR = os.getcwd()
MONO_PITCH_TRACKER_DIR = os.path.join(NOTEBOOK_DIR, '..')
MEDLEYDB_DIR = os.path.join(MONO_PITCH_TRACKER_DIR, 'medleydb/')

MONO_PITCH_TRACKER_DIR

'/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/notebooks/..'

In [5]:
import csv
import librosa
import pandas as pd
from pitch_tracker.utils.constants import SAMPLE_RATE, HOP_LENGTH

def create_melody1_annotation(audio_path:str, annotation_path: str, fs:int=SAMPLE_RATE, hop:int=HOP_LENGTH):
    """Create a melody1 annotation from pitch annotations
    """
    df = pd.read_csv(annotation_path, delim_whitespace=True, header=None)
    f0_annotation = df.values.tolist()
    
    # f0_annotation = []
    # with open(annotation_path, 'r') as fhandle:
    #     linereader = csv.reader(fhandle, delimiter='\t')
    #     for line in linereader:
    #         timestamp = float(line[0])
    #         f0 = float(line[1])
    #         f0_annotation.append((timestamp, f0))
    
    duration = librosa.get_duration(filename=audio_path)
    melody_sequence = make_blank_melody_sequence(duration, fs, hop)
    
    melody1 = add_sequence_to_melody(
        total_duration=duration,
        f0_sequence=f0_annotation,
        melody_sequence=melody_sequence,
        fs=fs,
        hop=hop,
    )

    return melody1

def write_melody1_label(melody1, output_path, overwrite:bool=True):
    if not os.path.exists(output_path) or overwrite:
        os.makedirs(os.path.split(output_path)[0], exist_ok=True)
        with open(output_path, "w", newline='') as fhandle:
            writer = csv.writer(fhandle)
            writer.writerows(melody1)
    else:
        print(f"Path: {output_path} already existed, skipping.")


def convert_to_new_label(audio_path:str, annotation_path:str, output_dir:str, fs:int=SAMPLE_RATE, hop=HOP_LENGTH):
    melody1 = create_melody1_annotation(audio_path, annotation_path, fs, hop)
    melody1_new = compute_end_time_stamps(melody1, convert_to_midi=True, round_method='round')

    audio_name = os.path.splitext(os.path.basename(audio_path))[0]
    label_output_path = os.path.join(output_dir, audio_name + '.csv')
    write_melody1_label(melody1_new, label_output_path)




In [6]:
MIREX05_PATH = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/datasets/mirex05TrainFiles/'
ADC2004_PATH = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/datasets/adc2004_full_set/'

for dataset_name, dataset_path in [('MIREX05',MIREX05_PATH), ('ADC2004',ADC2004_PATH)]:
    ref_paths =  sorted(list_all_file_paths_in_dir(dataset_path, exts='txt'))
    audio_paths = sorted(list_all_file_paths_in_dir(dataset_path, exts=('.mp3', '.wav')))
    out_dir = f'../content/gen_label_test/{dataset_name}/'
    
    os.makedirs(out_dir, exist_ok=True)
    for audio_path, ref_path in tqdm(zip(audio_paths, ref_paths)):
        try:
            convert_to_new_label(audio_path, ref_path, out_dir)
        except Exception as e:
            print(f'{ref_path} - {e}')

0it [00:00, ?it/s]/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:74

/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/datasets/adc2004_full_set/README.txt - Error tokenizing data. C error: Expected 3 fields in line 3, saw 6



/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
/Users/tien.d/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/librosa/core/convert.py:740: RuntimeWarning: divide by zero encountered in log2
  return 12 * (np.log2(np.asanyarray(frequencies)) - np.log2(440.0)) + 69
20it [00:00, 88.05it/s]


In [7]:
MIREX05_label_dir = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label_test/MIREX05'
ADC2004_label_dir = '/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/content/gen_label_test/ADC2004'
m05 = dataset.create_dataset_path_dict_ext(MIREX05_label_dir, MIREX05_PATH)
adc04 = dataset.create_dataset_path_dict_ext(ADC2004_label_dir, ADC2004_PATH)

In [22]:
from pitch_tracker.utils.constants import F_MIN, N_CLASS, N_FFT, PATCH_SIZE

def create_pickled_data(label_dir, audio_dir, output_pickled_dir_label, stft_hop_size, analysis_frame_size, n_mels):
    
    label_dict = dataset.create_label_dict_from_dir(label_dir)
    for k, v in label_dict.items():
        print(k, v.shape)
    
    dataset_paths = dataset.create_dataset_path_dict_ext(label_dir, audio_dir)
    analysis_frame_time = analysis_frame_size * (stft_hop_size/SAMPLE_RATE)
    patch_step = int(PATCH_SIZE/2)

    feature_label_gen = dataset.create_feature_label_generator(
        dataset_path_dict=dataset_paths,
        sample_rate=SAMPLE_RATE,
        n_fft=N_FFT,
        n_mels=n_mels,
        n_class=N_CLASS,
        hop_length=stft_hop_size,
        patch_step=patch_step,
        patch_size=PATCH_SIZE,
        analysis_frame_size=analysis_frame_size,
        analysis_frame_time=analysis_frame_time,
        dist_threshold=0.1,
        empty_threshold=0.3,
        fmin=F_MIN
    )

    output_pickled_dir = f'{MONO_PITCH_TRACKER_DIR}/content/pickled_database/{output_pickled_dir_label}/{stft_hop_size}_{analysis_frame_size}_{n_mels}/'
    passed_songs = dataset.write_feature_label_to_disk_by_frame(
        feature_label_gen, output_pickled_dir, categorize_by_subdir = True)
    failed_songs = [label for label in dataset_paths if label not in passed_songs]
    print(failed_songs)
    del failed_songs

In [23]:
create_pickled_data(ADC2004_label_dir, ADC2004_PATH,'ADC2004', 512, 5, 176)
create_pickled_data(MIREX05_label_dir, MIREX05_PATH,'MIREX05', 512, 5, 176)

midi4 (52, 3)
pop4 (125, 3)
midi1 (17, 3)
pop1 (413, 3)
midi2 (64, 3)
pop3 (167, 3)
midi3 (21, 3)
pop2 (253, 3)
opera_male5 (586, 3)
opera_fem4 (338, 3)
opera_male3 (310, 3)
opera_fem2 (52, 3)
jazz4 (12, 3)
daisy2 (41, 3)
daisy3 (57, 3)
jazz1 (38, 3)
jazz2 (33, 3)
daisy4 (38, 3)
jazz3 (29, 3)
[]


train09 (286, 3)
train08 (351, 3)
train13MIDI (77, 3)
train12 (43, 3)
train06 (209, 3)
train07 (329, 3)
train05 (215, 3)
train11 (72, 3)
train10 (60, 3)
train04 (497, 3)
train01 (185, 3)
train03 (261, 3)
train02 (478, 3)
[]
